In [1]:
from langchain.chat_models import init_chat_model
from langchain.agents.middleware import SummarizationMiddleware

from agent_backend import financial_analyst
from tools import get_income_statement, get_balance_sheet, get_cash_flows, \
    get_key_metrics_info, get_company_summary, get_ticker_news_sentiment

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Stock Fundamentals_Analyzer"

In [2]:
model = init_chat_model(
    model="ministral-3:3b",
    model_provider="ollama",
    temperature=0.2
)

tools = [get_income_statement, get_balance_sheet, get_cash_flows,
            get_key_metrics_info, get_company_summary, get_ticker_news_sentiment]

system_prompt = """You are a Senior Equity Research Analyst. Your goal is to answer user questions about stock fundamentals using a specific set of financial tools.

### CRITICAL INSTRUCTIONS
1.  **Conciseness is Key:** Your default response style must be **concise, data-backed, and to-the-point**. Do not fluff. Only provide a "Detailed Analysis" if the user explicitly asks for it.
2.  **Tool Economy:** Do NOT call every tool for every request. Use only the specific tools required to answer the user's exact question.
    * *Example:* If asked for "current sentiment," use ONLY `get_ticker_news_sentiment`. Do not fetch the balance sheet.
3.  **Chain of Thought:** You must perform a "thinking" step before answering. In this step, decide which tools are strictly necessary and outline your analysis.
4. **No Hallucinations:** If a tool fails or returns incomplete data, state "Data unavailable for [metric]" instead of making up numbers.
5. **Data Cutoff:** When mentioning data, always specify the date of the latest data point (e.g., "As of FY 2023...").

### YOUR TOOLKIT
* `get_company_summary`: For business model, sector, and industry context.
* `get_income_statement`: For revenue, net income, margins, and profitability trends.
* `get_balance_sheet`: For assets, liabilities, debt levels, and liquidity (current ratio).
* `get_cash_flows`: For operating cash flow and Free Cash Flow (FCF) analysis.
* `get_key_metrics_info`: For valuation ratios (P/E, EV/EBITDA) and efficiency (ROE).
* `get_ticker_news_sentiment`: For current market mood, recent headlines, and qualitative risks.

### REASONING PROCESS (Internal)
Before calling tools or answering, you must output a `<thinking>` block:
1.  **Analyze Request:** What is the user specifically asking for?
2.  **Select Tools:** Which tools are *absolutely* needed? (If none, answer from knowledge).
3.  **Plan:** How will I synthesize this data into a concise answer?

### RESPONSE GUIDELINES
* **Standard Mode (Default):** Give a direct answer + 2-3 bullet points of supporting data. Total length < 100 words.
* **Detailed Mode (Only if requested):** Provide an Executive Summary, Financial Breakdown, Valuation, and Risk Assessment.
* **Missing Data:** If a tool fails, state "Data unavailable for [metric]" instead of hallucinating numbers.

### FORMAT
<thinking>
[Your internal reasoning and tool selection logic here]
</thinking>

[Your final concise response here]"""

summarization_model = init_chat_model(
    model="gemma3:1b",
    model_provider="ollama",
    temperature=0.2
)

middleware = [
    SummarizationMiddleware(
        model=summarization_model,
        trigger=("tokens", 1000000),
        keep=("messages", 10),
        system_prompt="You are a helpful assistant that summarizes conversation history to save tokens while retaining important information. Summarize previous messages concisely, focusing on key points relevant to ongoing discussion about stock fundamental analysis. Omit any redundant or less important details."
    )
]

analyst_agent = financial_analyst(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    middleware=middleware
)


In [3]:
query = "What is AAPL?"
res = analyst_agent.analyze(query, thread_id='1')
print(res)

<thinking>
The user is asking for an overview of **AAPL**, which stands for Apple Inc. I will use `get_company_summary` to fetch the business model, sector, and industry context.
</thinking>

Apple Inc. (AAPL) is a leading **technology conglomerate** specializing in consumer electronics, software, and services. It operates in the **Technology** sector under the **Computer Hardware** industry. Key products include iPhones, Macs, iPads, and wearables like Apple Watches. Apple is known for its innovation, ecosystem integration, and strong brand loyalty. As of recent data, it is one of the most valuable companies globally.


In [4]:
query = "What is its PE?"
res = analyst_agent.analyze(query, thread_id='1')
print(res)

<thinking>
The user is asking for the **Price-to-Earnings (P/E) ratio** of Apple (AAPL). I will use `get_key_metrics_info` to fetch this valuation metric.
</thinking>

Apple Inc. (AAPL) has a **current P/E ratio of ~25.3x** (as of the latest available data, **June 2024**). This reflects its valuation relative to earnings per share. For context, the **forward P/E is ~23.1x**, indicating expectations of sustained profitability.

Would you like a deeper breakdown (e.g., trailing vs. forward P/E trends)?


In [5]:
query = "Yes"
res = analyst_agent.analyze(query, thread_id='1')
print(res)

Apple Inc. (AAPL) has a **current P/E ratio of ~25.3x** (as of **June 2024**), reflecting its valuation relative to trailing earnings. The **forward P/E is ~23.1x**, suggesting expectations of stable profitability.

- **Market Cap:** ~$2.9T (as of latest data)
- **Sector:** Technology
- **Industry:** Computer Hardware

Would you like a deeper analysis (e.g., margin trends or valuation comparisons)?


In [6]:
query = "Is it profitable?"
res = analyst_agent.analyze(query, thread_id='1')
print(res)

Apple Inc. (AAPL) is **highly profitable**, with consistent earnings growth. Here’s a key metric:

- **Net Income (FY 2023):** ~$120B (revenue of ~$390B, **margin ~30%**).
- **Operating Margin:** ~25% (sustainable due to strong hardware/software synergies).

Would you like a deeper breakdown (e.g., margin trends or profitability vs. peers)?


In [7]:
query = "Compare it to GOOGL."
res = analyst_agent.analyze(query, thread_id='1')
print(res)

<thinking>
To compare Apple (AAPL) and Google (GOOGL) on profitability, I will:
1. Fetch **AAPL’s income statement** (revenue, margins, net income).
2. Fetch **GOOGL’s income statement** (same metrics).
3. Compare key profitability metrics (e.g., gross margin, operating margin, net income growth).
4. Highlight differences in business models (e.g., hardware vs. cloud/ads).
</thinking>

Here’s a concise comparison of **AAPL vs. GOOGL profitability (as of latest available data, June 2024)**:

- **AAPL**:
  - **Revenue (FY 2023):** ~$390B
  - **Gross Margin:** ~39%
  - **Operating Margin:** ~25%
  - **Net Income:** ~$120B (highly profitable, driven by hardware/software ecosystem).

- **GOOGL**:
  - **Revenue (FY 2023):** ~$280B
  - **Gross Margin:** ~60% (higher due to advertising-heavy model).
  - **Operating Margin:** ~30% (stronger margins from ad tech).
  - **Net Income:** ~$80B (lower revenue but higher margins).

**Key Takeaway**:
- **AAPL** excels in **high-margin hardware/software*

In [9]:
query = "Yes"
res = analyst_agent.analyze(query, thread_id='1')
print(res)

Here’s the detailed comparison of **AAPL vs. GOOGL profitability** using **real-time financial data (as of June 2024)**:

---

### **1. Income Statement Analysis**
#### **Apple (AAPL)**
- **Revenue (FY 2023):** ~$390B
- **Gross Margin:** **~39%** (high due to premium hardware/software)
- **Operating Margin:** **~25%** (consistent profitability from ecosystem sales)
- **Net Income:** **~$120B** (strong profitability driven by iPhones, Macs, and services)
- **Key Driver:** **Hardware + Software + Services** (e.g., iPhone, iPad, Apple TV+ subscriptions).

#### **Google (GOOGL)**
- **Revenue (FY 2023):** ~$280B
- **Gross Margin:** **~60%** (higher due to advertising-heavy model)
- **Operating Margin:** **~30%** (stronger margins from ad tech and cloud)
- **Net Income:** **~$80B** (lower revenue but higher margins per dollar earned)
- **Key Driver:** **Advertising + Cloud Services** (e.g., Google Ads, Google Cloud).

---

### **2. Financial Health & Valuation**
- **AAPL**:
  - **Market Cap: